# Task 1

In [1]:
! pip install lxml
import pandas as pd

1st read the Table. Actually the page contains many tables, but the required one is the first, therefore the [0] after the function call.

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

Now the columns have to be renamed according to the task.

In [3]:
df.rename(columns={"Postcode": "PostalCode", "Neighbourhood":"Neighborhood"}, inplace=True)

Next drop all rows where Borough is not assigned.

In [4]:
df = df[df.Borough != 'Not assigned']

So now the index is a bit messed up. But this can easily be changed:

In [5]:
df = df.reset_index(drop=True)

This next line is a bit more tricky. First we group the data by PostalCode and Borough, but we do not want to set them as index. We then Join the Neighborhoods into one string. Then we reset the index again, because it got messed up.
Last but not least, the joined column is renamed back to the name "Neighborhood".

In [6]:
df = pd.DataFrame(df.groupby(['PostalCode','Borough'], as_index=False)["Neighborhood"].apply(lambda x: ', '.join(x))).reset_index().rename(columns={0:"Neighborhood"})

If there are any rows, where the Neighborhood is not assigned, we just use the value from Borough:

In [7]:
df["Neighborhood"][df.Neighborhood == 'Not assigned'] = df.Borough[df.Neighborhood == 'Not assigned'] 

In [8]:
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [9]:
df.shape

(103, 3)

# Task 2

The geocoder tool did not work, so I had to use the csv file.

In [23]:
coord_df = pd.read_csv("https://cocl.us/Geospatial_data")

In [26]:
coord_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

The only thing that has to be done for this task is to merge both datasets on the PostalCode key

In [39]:
df = pd.merge(df, coord_df, on="PostalCode")

In [40]:
df

,PostalCode,Borough,Neighborhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,43.773136,-79.239476
...,...,...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724,43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437,43.739416,-79.588437


# Task 3

In [41]:
import shapely

ModuleNotFoundError: No module named 'shapely'